In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.metrics import r2_score

Dataframes

In [ ]:
eci_customer_data = pd.read_csv('data/eci_customer_data.csv')
eci_product_groups = pd.read_csv('data/eci_product_groups.csv')
eci_product_master = pd.read_csv('data/eci_product_master.csv')
eci_stores_clusters = pd.read_csv('data/eci_stores_clusters.csv')
eci_stores_data = pd.read_csv('data/eci_stores.csv')
eci_transactions = pd.read_csv('data/eci_transactions.csv')

## Análisis exploratorio de datos

In [ ]:
eci_customer_data.head()

,client_id,phone_number,email_address,city,state,zip_code,education_level,occupation,loyalty_member,loyalty_number,loyalty_points
0,500001,NaN,n.edwards+spam@outlook.com,Spokane,WA,99201,High School,NaN,No,NaN,339
1,500002,(412) 698-7960,lisa_cook@yahoo.com,Pittsburgh,PA,15222,High School,Sales Representative,Yes,LP525082,406
2,500003,(303) 491-3774,NaN,Fort Collins,CO,80526,College Graduate,Analyst,True,950139,14
3,500004,(803) 255-0104,betty-campbell@gmail.com,Charleston,SC,29401,Some College,Supervisor,No,NaN,NaN
4,500005,(303) 596-8987,NaN,Aurora,CO,80012,Associates,NaN,Yes,NaN,NaN


In [ ]:
eci_customer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801923 entries, 0 to 801922
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   client_id        801923 non-null  int64 
 1   phone_number     705749 non-null  object
 2   email_address    657079 non-null  object
 3   city             801923 non-null  object
 4   state            801923 non-null  object
 5   zip_code         734663 non-null  object
 6   education_level  682042 non-null  object
 7   occupation       657407 non-null  object
 8   loyalty_member   762074 non-null  object
 9   loyalty_number   440681 non-null  object
 10  loyalty_points   392749 non-null  object
dtypes: int64(1), object(10)
memory usage: 67.3+ MB


In [ ]:
# Resumen estadístico de eci_customer_data
eci_customer_data.describe(include='all')

,client_id,phone_number,email_address,city,state,zip_code,education_level,occupation,loyalty_member,loyalty_number,loyalty_points
count,8.019230e+05,705749,657079,801923,801923,734663,682042,657407,762074,440681,392749
unique,NaN,684761,418024,108,22,341,27,38,8,386689,27254
top,NaN,555-1234,@gmail.com,Columbus,FL,43229,Bachelor's Degree,Manager,Yes,PENDING,9999999
freq,NaN,3486,2365,18116,50144,5259,109273,49897,267105,3804,1292
mean,1.010913e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,2.950790e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,5.000010e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,7.551395e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.010748e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.266510e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# Chequeamos missings
eci_customer_data.isnull().sum()  

client_id               0
phone_number        96174
email_address      144844
city                    0
state                   0
zip_code            67260
education_level    119881
occupation         144516
loyalty_member      39849
loyalty_number     361242
loyalty_points     409174
dtype: int64

In [28]:
print((eci_customer_data.isna().mean()).round(2))

client_id          0.00
phone_number       0.12
email_address      0.18
city               0.00
state              0.00
zip_code           0.08
education_level    0.15
occupation         0.18
loyalty_member     0.05
loyalty_number     0.45
loyalty_points     0.51
dtype: float64


In [ ]:
for col in eci_customer_data.columns:
    print(col, eci_customer_data[col].nunique())

client_id 801923
phone_number 684761
email_address 418024
city 108
state 22
zip_code 341
education_level 27
occupation 38
loyalty_member 8
loyalty_number 386689
loyalty_points 27254


In [33]:
print(eci_customer_data['loyalty_member'].unique())

['No' 'Yes' 'True' '1' nan 'N' 'Y' 'False' '0']


In [8]:
eci_product_groups.head()

,sku,product_name,price_group_id,price_group_name,group_type
0,BOOEDTE001,PageTurn Essential Textbooks,BACK_TO_SCHOOL_01,Back to School Essentials,Seasonal
1,BOOEDTE002,Bookworm Advanced Textbooks,BACK_TO_SCHOOL_01,Back to School Essentials,Seasonal
2,BOOEDTE004,PageTurn Essential Textbooks,BACK_TO_SCHOOL_01,Back to School Essentials,Seasonal
3,CLOMESH002,DressRight Shirts Collection,BACK_TO_SCHOOL_01,Back to School Essentials,Seasonal
4,CLOMESH005,StyleX Relaxed Shirts,BACK_TO_SCHOOL_01,Back to School Essentials,Seasonal


In [9]:
eci_product_groups.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   sku               80 non-null     object
 1   product_name      80 non-null     object
 2   price_group_id    80 non-null     object
 3   price_group_name  80 non-null     object
 4   group_type        80 non-null     object
dtypes: object(5)
memory usage: 3.3+ KB


In [10]:
for col in eci_product_groups.columns:
    print(col, eci_product_groups[col].nunique())

sku 73
product_name 70
price_group_id 8
price_group_name 8
group_type 5


In [38]:
print(eci_product_groups['price_group_id'].unique())

['BACK_TO_SCHOOL_01' 'CASUAL_BOTTOMS_01' 'COMP_SETUP_01'
 'FITNESS_COMBO_01' 'GIFT_UNDER_50_01' 'MOBILE_DEVICES_01'
 'MOVIE_NIGHT_01' 'WELLNESS_BUNDLE_01']


In [39]:
print(eci_product_groups['price_group_name'].unique())

['Back to School Essentials' 'Casual Bottoms' 'Computer Setup Essentials'
 'Complete Fitness Package' 'Gift Ideas Under $50'
 'Personal Computing Devices' 'Movie Night Snacks' 'Wellness & Self-Care']


In [37]:
print(eci_product_groups['group_type'].unique())

['Seasonal' 'Substitute' 'Bundle' 'Promotional' 'Lifestyle']


In [11]:
eci_product_master.head()

,sku,product_name,category,group,subgroup,brand,base_price,initial_ticket_price,costos
0,ELECOLA001,TechPro Laptops Smart 2025,Electronics,Computing,Laptops,TechPro,284.94,355.14,225.84
1,ELECOLA002,Electra Laptops Ultra Pro,Electronics,Computing,Laptops,Electra,311.83,352.74,209.99
2,ELECOLA003,CircuitCity Laptops Air 2024,Electronics,Computing,Laptops,CircuitCity,246.96,301.35,192.61
3,ELECOLA004,TechPro Laptops Elite 2025,Electronics,Computing,Laptops,TechPro,324.96,420.49,212.23
4,ELECOLA005,CircuitCity Laptops Pro S,Electronics,Computing,Laptops,CircuitCity,339.88,386.23,231.75


In [12]:
eci_product_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 861 entries, 0 to 860
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   sku                   861 non-null    object 
 1   product_name          861 non-null    object 
 2   category              861 non-null    object 
 3   group                 861 non-null    object 
 4   subgroup              861 non-null    object 
 5   brand                 861 non-null    object 
 6   base_price            861 non-null    float64
 7   initial_ticket_price  861 non-null    float64
 8   costos                861 non-null    float64
dtypes: float64(3), object(6)
memory usage: 60.7+ KB


In [13]:
for col in eci_product_master.columns:
    print(col, eci_product_master[col].nunique())

sku 854
product_name 713
category 8
group 23
subgroup 74
brand 40
base_price 801
initial_ticket_price 818
costos 770


In [41]:
print(eci_product_master['category'].unique())

['Electronics' 'Clothing' 'Home & Kitchen' 'Sports' 'Books' 'Beauty'
 'Groceries' 'Toys']


In [14]:
eci_stores_clusters.head()

,STORE_ID,BRAND,STORE_NAME,CLUSTER
0,S00001,PrimeGoods,PrimeGoods - Hillsboro,Cluster_West_Coast
1,S00002,MegaShop,MegaShop Indianapolis Eastside,Midwest_Cluster
2,STORE0003,MarketExpress,MarketExpress - Las Vegas,Mountain_Cluster
3,STORE0004,MegaShop,MegaShop - Oklahoma City,Texas_Cluster
4,S00005,PrimeGoods,PrimeGoods - Charlotte,NC_Cluster


In [15]:
eci_stores_clusters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157 entries, 0 to 156
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   STORE_ID    157 non-null    object
 1   BRAND       157 non-null    object
 2   STORE_NAME  157 non-null    object
 3   CLUSTER     140 non-null    object
dtypes: object(4)
memory usage: 5.0+ KB


In [16]:
for col in eci_stores_clusters.columns:
    print(col, eci_stores_clusters[col].nunique())

STORE_ID 157
BRAND 9
STORE_NAME 144
CLUSTER 13


In [17]:
eci_stores_data.head()

,STORE_ID,BRAND,STORE_NAME,ADDRESS1,ADDRESS2,CITY,STATE,ZIP,OPENDATE,CLOSEDATE,STORE_TYPE,REGION
0,S00001,PrimeGoods,PrimeGoods - Hillsboro,5735 NE Washington Pl,NaN,Hillsboro,OR,74820,2015-01-10 00:00:00,NaN,Express,West
1,S00002,MegaShop,MegaShop Indianapolis Eastside,3557 Cedar Ct,Unit 89,Indianapolis,IN,33483-1775,2012-03-23 00:00:00,NaN,Express,Midwest
2,STORE0003,MarketExpress,MarketExpress - Las Vegas,7870 W Church Ave,NaN,Las Vegas,NV,49099,2021-09-23 00:00:00,NaN,Outlet,West
3,STORE0004,MegaShop,MegaShop - Oklahoma City,3105 N Washington Pl,NaN,Oklahoma City,OK,80592,06/30/2015,NaN,Mall,Southwest
4,S00005,PrimeGoods,PrimeGoods - Charlotte,8756 SW Maple Ln,Suite 848,Charlotte,NC,32299,2019-06-10 00:00:00,NaN,Express,Southeast


In [18]:
eci_stores_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157 entries, 0 to 156
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   STORE_ID    157 non-null    object
 1   BRAND       157 non-null    object
 2   STORE_NAME  157 non-null    object
 3   ADDRESS1    157 non-null    object
 4   ADDRESS2    43 non-null     object
 5   CITY        157 non-null    object
 6   STATE       157 non-null    object
 7   ZIP         151 non-null    object
 8   OPENDATE    157 non-null    object
 9   CLOSEDATE   18 non-null     object
 10  STORE_TYPE  155 non-null    object
 11  REGION      157 non-null    object
dtypes: object(12)
memory usage: 14.8+ KB


In [19]:
for col in eci_stores_data.columns:
    print(col, eci_stores_data[col].nunique())

STORE_ID 157
BRAND 9
STORE_NAME 144
ADDRESS1 157
ADDRESS2 34
CITY 77
STATE 22
ZIP 151
OPENDATE 155
CLOSEDATE 18
STORE_TYPE 8
REGION 5


In [20]:
eci_transactions.head()

,TRANSACTION_ID,DATE,STORE_ID,SKU,QUANTITY,PRICE,TOTAL_SALES,SUBGROUP,STORE_SUBGROUP_DATE_ID
0,17185,2021-01-01,S00148,CLOMEJA001,1.0,52.80,52.80,Jackets,S00148_Jackets_2021-01-01
1,4108,2021-01-01,S00035,CLOCHIN014,1.0,55.25,55.25,Infants,S00035_Infants_2021-01-01
2,2400,2021-01-01,S00020,BEAHASH005,1.0,40.19,40.19,Shampoo,S00020_Shampoo_2021-01-01
3,7978,2021-01-01,S00068,TOYGABO004,1.0,35.41,35.41,Board Games,S00068_Board Games_2021-01-01
4,6946,2021-01-01,S00059,BEAHASH006,1.0,38.28,38.28,Shampoo,S00059_Shampoo_2021-01-01


In [21]:
eci_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19004759 entries, 0 to 19004758
Data columns (total 9 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   TRANSACTION_ID          int64  
 1   DATE                    object 
 2   STORE_ID                object 
 3   SKU                     object 
 4   QUANTITY                float64
 5   PRICE                   float64
 6   TOTAL_SALES             float64
 7   SUBGROUP                object 
 8   STORE_SUBGROUP_DATE_ID  object 
dtypes: float64(3), int64(1), object(5)
memory usage: 1.3+ GB


In [22]:
for col in eci_transactions.columns:
    print(col, eci_transactions[col].nunique())

TRANSACTION_ID 19004759
DATE 1095
STORE_ID 152
SKU 854
QUANTITY 55
PRICE 37291
TOTAL_SALES 152992
SUBGROUP 73
STORE_SUBGROUP_DATE_ID 5206793
